# Make Slack Less Noisy 🔇

This notebook allows us to categorize and sum counts of automated workflow messages.  The goal is to identify and reduce frequent notifications leading to less noise and less notification blindness

In [1]:
import pandas as pd
import ast
import re
from datetime import datetime
from pathlib import Path
import json

In [7]:
START_DATE="2024-12-01"
NOW_DATE=datetime.now().strftime('%Y-%m-%d')

In [8]:
# Retrieved from slack admin console
# https://lightkeeperdev.slack.com/services/export

#directory = Path('./export/update_errors/')
directory = Path('C:/Users/SamMiller/Downloads/update_errors')
pattern = re.compile(r'\d{4}-\d{2}-\d{2}')
messages = []
files = [file for file in directory.glob('*.json') if pattern.search(file.stem)]
for file in files:
    with open(file, 'r') as f:
        data = json.load(f)
        if isinstance(data, list):
            messages.extend(data)

len(messages)

2763

### Clean the data

In [9]:
def clean_message(message):
    clean = {}
    if not message.get('attachments'):
        return
    customer_search = re.search('http://(.*)\.lightkeeperhq.com\|.*',  message['attachments'][0]['title'])
    customer = customer_search.group(1)
    head = message['text']
    body = message['attachments'][0]['text']
    dt = datetime.fromtimestamp(int(float(message['ts'])))
    isAtChannel = '<!channel>' in head
    
    return {
     'customer': customer,
     'head': head,
     'body': body,
     'datetime': dt,
     'isAtChannel': isAtChannel
    }

<>:5: SyntaxWarning: invalid escape sequence '\.'
<>:5: SyntaxWarning: invalid escape sequence '\.'
C:\Users\SamMiller\AppData\Local\Temp\ipykernel_43220\1629103741.py:5: SyntaxWarning: invalid escape sequence '\.'
  customer_search = re.search('http://(.*)\.lightkeeperhq.com\|.*',  message['attachments'][0]['title'])


In [10]:
clean_messages = [clean_message(message) for message in messages]
clean_messages = [message for message in clean_messages if message]

In [11]:
clean_messages[1]

{'customer': 'lonepine',
 'head': 'T0 ArchiveTask has not completed for 2024-12-31',
 'body': '```An update will be sent once it has completed.```',
 'datetime': datetime.datetime(2025, 1, 1, 0, 0, 31),
 'isAtChannel': False}

In [12]:
len(clean_messages)

2756

In [13]:
df = pd.DataFrame(clean_messages)

In [14]:
df.dtypes

customer               object
head                   object
body                   object
datetime       datetime64[ns]
isAtChannel              bool
dtype: object

### Filter and enhance

In [15]:
# Filter by datetime
df = df.loc[(df['datetime'] > START_DATE) & (df['datetime'] < NOW_DATE)]

In [16]:
# Create a map of message types we can aggregate by
MESSAGE_TYPES = {
    'daily_update_sla_fail': '^<!channel>.*Daily.*has not completed.*',
    't0_update_sla_fail': '^<!channel>.*T0.*has not completed.*',
    'broker_update_sla_fail': '.*Broker UpdateTask has not completed.*',
    'daily_missing_files': '^Daily files missing for.*',
    'daily_received_files': '^All Daily required files have now been receive.*',
    't0_missing_files': '^T0 files missing for.*',
    't0_received_files': '^All T0 required files have now been received.*',
    'build_error': '.*\.build error.*',
    'daily_non_current': '.*Daily UpdateTask.build-confirm non-current.*',
    't0_non_current': '.*T0 UpdateTask.build-confirm non-current.*',
    'daily_update_sla_recovery': 'Daily.*has now completed.*',
    't0_update_sla_recovery': 'T0.*has now completed.*',
}

df['message_type'] = df['head'].apply(lambda x: ','.join([k for k, v in MESSAGE_TYPES.items() if re.match(v, x)]) or None)

<>:10: SyntaxWarning: invalid escape sequence '\.'
<>:10: SyntaxWarning: invalid escape sequence '\.'
C:\Users\SamMiller\AppData\Local\Temp\ipykernel_43220\3151063467.py:10: SyntaxWarning: invalid escape sequence '\.'
  'build_error': '.*\.build error.*',


In [17]:
df = df.dropna()

In [18]:
counts = df.groupby(['message_type','customer'])['head'].count()\
                    .reset_index().sort_values(['head'], ascending=False)
countsAtChannel = df.loc[df['isAtChannel']==True].groupby(['message_type','customer'])['head'].count()\
                    .reset_index().sort_values(['head'], ascending=False)

### @channel counts

In [19]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  
    display(countsAtChannel)

,message_type,customer,head
169,t0_update_sla_fail,tremblant,31
158,t0_update_sla_fail,hudsonbay,27
4,build_error,altimeter,16
30,build_error,cypressfunds,13
100,daily_update_sla_fail,cypressfunds,13
156,t0_update_sla_fail,contour,9
27,build_error,corvex,9
89,daily_update_sla_fail,altimeter,8
147,t0_update_sla_fail,altimeter,8
3,build_error,altapark,8


### Non-alerting workflow counts

In [20]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  
    display(counts)

,message_type,customer,head
278,t0_missing_files,tremblant,71
321,t0_received_files,tremblant,69
318,t0_received_files,slatepath,68
275,t0_missing_files,slatepath,65
373,t0_update_sla_recovery,tremblant,53
365,t0_update_sla_recovery,hudsonbay,52
323,t0_received_files,vivaldi,48
308,t0_received_files,hudsonbay,45
313,t0_received_files,lonepine,35
305,t0_received_files,engle,33


### Latest Daily Recovery Times

In [21]:
dailyRecoveryMessages = df[df['message_type'] == 'daily_update_sla_recovery'] 
latestTimes = dailyRecoveryMessages.groupby(['customer', 'message_type'])['datetime'].max().reset_index()
display(latestTimes)

,customer,message_type,datetime
0,altimeter,daily_update_sla_recovery,2025-04-04 09:44:57
1,beta-1-aventail,daily_update_sla_recovery,2025-01-07 11:46:48
2,beta-tremblant,daily_update_sla_recovery,2025-01-02 09:10:34
3,corvex,daily_update_sla_recovery,2025-02-12 11:22:49
4,crestline,daily_update_sla_recovery,2025-04-09 12:47:56
5,cypressfunds,daily_update_sla_recovery,2025-03-03 09:52:56
6,eagle,daily_update_sla_recovery,2025-03-11 09:02:48
7,fortress,daily_update_sla_recovery,2025-03-06 13:11:49
8,goldentree,daily_update_sla_recovery,2025-02-03 08:49:30
9,gsam,daily_update_sla_recovery,2025-01-01 10:14:01


### Latest T0 Recovery Times

In [22]:
dailyRecoveryMessages = df[df['message_type'] == 't0_update_sla_recovery'] 
latestTimes = dailyRecoveryMessages.groupby(['customer', 'message_type'])['datetime'].max().reset_index()
display(latestTimes)

,customer,message_type,datetime
0,adage,t0_update_sla_recovery,2025-01-06 23:38:09
1,adelphi,t0_update_sla_recovery,2025-04-01 20:00:25
2,altimeter,t0_update_sla_recovery,2025-04-04 10:04:20
3,anomaly,t0_update_sla_recovery,2025-02-14 19:06:51
4,aregence,t0_update_sla_recovery,2025-01-27 21:04:29
5,avalonglobal,t0_update_sla_recovery,2025-03-24 00:00:33
6,beta-1-aventail,t0_update_sla_recovery,2025-01-02 20:25:31
7,braidwell,t0_update_sla_recovery,2025-04-07 18:32:41
8,contour,t0_update_sla_recovery,2025-03-14 21:04:07
9,conversant,t0_update_sla_recovery,2025-01-13 17:30:31
